<a href="https://colab.research.google.com/github/luntrum/model_predict_age_gender_and_faceshape/blob/main/agegender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import math as m
import math
import keras_preprocessing
import seaborn as sns
import pandas as pd

from PIL import Image, ImageOps

from keras import optimizers
from keras.models import Model, Sequential,load_model
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation,Conv2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.models import model_from_json
from tensorflow.keras.optimizers import Adam 


from sklearn.model_selection import train_test_split


from os import listdir

In [ ]:
images = []
ages = []
genders = []

for i in os.listdir('/content/gdrive/MyDrive/Colab Notebooks/cuoiki/ageandgender/')[0:8000]:
    split = i.split('_')
    ages.append(int(split[0]))
    genders.append(int(split[1]))
    images.append(Image.open('/content/gdrive/MyDrive/Colab Notebooks/cuoiki/ageandgender/' + i))

In [ ]:
images = pd.Series(list(images), name = 'Images')
ages = pd.Series(list(ages), name = 'Ages')
genders = pd.Series(list(genders), name = 'Genders')

df = pd.concat([images, ages, genders], axis=1)
df = df[df['Ages'] >= 10]
df = df[df['Ages'] <= 60]

df

,Images,Ages,Genders
0,<PIL.JpegImagePlugin.JpegImageFile image mode=...,37,0
1,<PIL.JpegImagePlugin.JpegImageFile image mode=...,37,0
2,<PIL.JpegImagePlugin.JpegImageFile image mode=...,37,0
3,<PIL.JpegImagePlugin.JpegImageFile image mode=...,37,0
4,<PIL.JpegImagePlugin.JpegImageFile image mode=...,37,0
...,...,...,...
4942,<PIL.JpegImagePlugin.JpegImageFile image mode=...,13,1
4943,<PIL.JpegImagePlugin.JpegImageFile image mode=...,14,0
4944,<PIL.JpegImagePlugin.JpegImageFile image mode=...,13,1
4945,<PIL.JpegImagePlugin.JpegImageFile image mode=...,14,1


In [ ]:
x = []
y = []
for i in range(len(df)):
    df['Images'].iloc[i] = df['Images'].iloc[i].resize((200,200), Image.ANTIALIAS)
    ar = np.asarray(df['Images'].iloc[i])
    x.append(ar)
    agegen = [int(df['Ages'].iloc[i]), int(df['Genders'].iloc[i])]
    y.append(agegen)
x = np.array(x)


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
y_age = df['Ages']
y_gender = df['Genders']
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(x, y_age, test_size=0.3, stratify=y_age)
x_train_gender, x_test_gender, y_train_gender, y_test_gender = train_test_split(x, y_gender, test_size=0.3, stratify=y_gender)

In [ ]:
agemodel = Sequential()
agemodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(64, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Conv2D(128, (3,3), activation='relu'))
agemodel.add(MaxPooling2D((2,2)))
agemodel.add(Flatten())
agemodel.add(Dense(64, activation='relu'))
agemodel.add(Dropout(0.5))
agemodel.add(Dense(1, activation='relu'))

agemodel.compile(loss='mean_squared_error',
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
             metrics=['accuracy'])

genmodel = Sequential()
genmodel.add(Conv2D(32, (3,3), activation='relu', input_shape=(200, 200, 3)))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(64, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Conv2D(128, (3,3), activation='relu'))
genmodel.add(MaxPooling2D((2,2)))
genmodel.add(Flatten())
genmodel.add(Dense(64, activation='relu'))
genmodel.add(Dropout(0.5))
genmodel.add(Dense(1, activation='sigmoid'))

genmodel.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
             metrics=['accuracy'])



In [ ]:
datagen = ImageDataGenerator(
      rescale=1./255., 
      width_shift_range = 0.1, 
      height_shift_range = 0.1, 
      horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale=1./255)

train1 = datagen.flow(x_train_age, 
                      y_train_age, 
                      batch_size=32)

test1 = test_datagen.flow(
        x_test_age, y_test_age,
        batch_size=32)



train2 = datagen.flow(x_train_gender, y_train_gender, batch_size=64)

test2 = test_datagen.flow(
        x_test_gender, y_test_gender,
        batch_size=64)




In [ ]:
history1 = agemodel.fit(train1, epochs=50, shuffle=True, validation_data=test1)

Epoch 1/50
109/109 [==============================] - 44s 274ms/step - loss: 385.0648 - accuracy: 0.0000e+00 - val_loss: 254.5471 - val_accuracy: 0.0000e+00
Epoch 2/50
109/109 [==============================] - 29s 266ms/step - loss: 288.2099 - accuracy: 0.0000e+00 - val_loss: 234.5704 - val_accuracy: 0.0000e+00
Epoch 3/50
109/109 [==============================] - 29s 266ms/step - loss: 274.6200 - accuracy: 0.0000e+00 - val_loss: 258.1729 - val_accuracy: 0.0000e+00
Epoch 4/50
109/109 [==============================] - 29s 265ms/step - loss: 274.2482 - accuracy: 0.0000e+00 - val_loss: 217.3980 - val_accuracy: 0.0000e+00
Epoch 5/50
109/109 [==============================] - 29s 267ms/step - loss: 246.3473 - accuracy: 0.0000e+00 - val_loss: 196.5410 - val_accuracy: 0.0000e+00
Epoch 6/50
109/109 [==============================] - 29s 266ms/step - loss: 241.3343 - accuracy: 0.0000e+00 - val_loss: 183.6979 - val_accuracy: 0.0000e+00
Epoch 7/50
109/109 [==============================] - 29s 

In [ ]:
history2 = genmodel.fit(train2, epochs=50, shuffle=True, validation_data=test2)

Epoch 1/50
55/55 [==============================] - 32s 532ms/step - loss: 0.6548 - accuracy: 0.6179 - val_loss: 0.5818 - val_accuracy: 0.7077
Epoch 2/50
55/55 [==============================] - 28s 508ms/step - loss: 0.5878 - accuracy: 0.7060 - val_loss: 0.5482 - val_accuracy: 0.7178
Epoch 3/50
55/55 [==============================] - 28s 511ms/step - loss: 0.5631 - accuracy: 0.7192 - val_loss: 0.4909 - val_accuracy: 0.7717
Epoch 4/50
55/55 [==============================] - 28s 508ms/step - loss: 0.5227 - accuracy: 0.7553 - val_loss: 0.4605 - val_accuracy: 0.8000
Epoch 5/50
55/55 [==============================] - 28s 509ms/step - loss: 0.5056 - accuracy: 0.7695 - val_loss: 0.4553 - val_accuracy: 0.8114
Epoch 6/50
55/55 [==============================] - 28s 510ms/step - loss: 0.4815 - accuracy: 0.7825 - val_loss: 0.4396 - val_accuracy: 0.8061
Epoch 7/50
55/55 [==============================] - 28s 509ms/step - loss: 0.4688 - accuracy: 0.7863 - val_loss: 0.4170 - val_accuracy: 0.8189

In [ ]:
agemodel.save('/content/gdrive/MyDrive/Colab Notebooks/cuoiki/agemodel.h')
genmodel.save('/content/gdrive/MyDrive/Colab Notebooks/cuoiki/gendermodel.h')


INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/cuoiki/agemodel.h/assets
INFO:tensorflow:Assets written to: /content/gdrive/MyDrive/Colab Notebooks/cuoiki/gendermodel.h/assets
